In [1]:
import numpy as np
from astropy.io import fits
from scipy.ndimage import rotate
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import aplpy
from matplotlib.patches import Rectangle
from astropy.visualization import (MinMaxInterval, AsinhStretch)
from astropy.wcs import WCS
from astropy.visualization import wcsaxes
from astropy import units as u
import pandas as pd
from astropy.table import Table, hstack
from astropy.table import join
import os

ImportError: cannot import name 'update_default_config' from 'astropy.config.configuration' (/home/luis/anaconda3/envs/my-env/lib/python3.9/site-packages/astropy/config/configuration.py)

In [ ]:
# Spectra VLT
df_spec = pd.read_csv("../flux-around-halpha.csv")

In [ ]:
# Convert Pandas DataFrame 'df' to Astropy Table
table_spc = Table.from_pandas(df_spec)
table_spc

In [ ]:
# Spectra VLT
table_img = Table.read("../flux-pseudoslit-halpha.ecsv", format="ascii.ecsv")

In [ ]:
table_img

In [ ]:
# Renaming the columns in table_spc to match table_img
table_spc.rename_column('Extension Name', 'ID')
table_img.rename_column('Name spec', 'FileID')
table_img["FileID"]

In [ ]:
# Convert Astropy tables to Pandas DataFrames
df1 = table_spc.to_pandas()
df2 = table_img.to_pandas()

In [ ]:
df1

In [ ]:
#Assuming merged_df3 is your DataFrame
constant_error = 0.2 * df2["Flux_slit"]  # 20% of Flux_slit
df2["Constant_Err"] = constant_error  # Assign the calculated error to a new column

In [ ]:
df2

In [ ]:
# Merge DataFrames on common columns
merged_df = pd.merge(df1, df2, left_on='ID', right_on='FileID', how='inner')

In [ ]:
merged_df

In [ ]:
# Sample data
x = np.linspace(0, 10, 100)  # Generate 100 points between 0 and 10
y = x  # Identity line: y equals x

In [ ]:
plt.plot(merged_df["Sum Fluxes"], merged_df["Flux_slit"], "o")
# Create a plot
plt.plot(x, y, label='Identity Line', color='red')
plt.xlim(0.55e-13, 14e-13)
plt.ylim(0.55e-13, 14e-13)

# Only the Halpha and [N II] lines

In [ ]:
# Spectra VLT
table_spec = Table.read("../flux-halpha-Nii.ecsv", format="ascii.ecsv")

In [ ]:
table_spec

In [ ]:
df3 = table_spec.to_pandas()

In [ ]:
# Merge DataFrames on common columns
merged_df3 = pd.merge(df3, df2, left_on='Extension Name', right_on='FileID', how='inner')

In [ ]:
merged_df3

In [ ]:
plt.plot(merged_df3["Fluxes"], merged_df3["Flux_slit"], "o")
# Create a plot
plt.plot(x, y, label='Identity Line', color='red')
plt.xlim(0.55e-13, 14e-13)
plt.ylim(0.55e-13, 14e-13)
plt.xlabel(r'F_spec(H$\alpha$ + [NII])')
plt.ylabel(r'F_splus(H$\alpha$ + [NII])')

In [ ]:
# Assuming merged_df3["Fluxes"] and merged_df3["Flux_slit"] are your measurements
differences = merged_df3["Flux_slit"] - merged_df3["Fluxes"]
means = (merged_df3["Flux_slit"] + merged_df3["Fluxes"]) / 2

# Create a figure with three subplots: scatter plot, Bland-Altman plot, and histogram
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Scatter plot with error bars
ax1.scatter(
    merged_df3["Fluxes"], merged_df3["Flux_slit"],
    color='skyblue',
    edgecolors='darkblue',
    alpha=0.7,  # Set transparency
    s=150,  # Set marker size
    label='PNe',
)
ax1.errorbar(
    merged_df3["Fluxes"], merged_df3["Flux_slit"],
    xerr=merged_df3["Err Fluxes"],
    fmt='none',  # No markers for error bars
    ecolor='gray',  # Error bar color
    capsize=5,  # Size of error bar caps
)
ax1.plot([-3, 3], [-3, 3], color='red', linestyle='--', label='Identity Line')
ax1.set_xlabel(r'F(H$\alpha$ + [NII]), Spec (VLT)', fontsize=15)
ax1.set_ylabel(r'F (H$\alpha$ + [NII]), SPLUS', fontsize=15)
ax1.set_xlim(0.55e-13, 14e-13)
ax1.set_ylim(0.55e-13, 14e-13)
ax1.tick_params(axis='x', labelsize=15) 
ax1.tick_params(axis='y', labelsize=15)
ax1.legend(fontsize="x-large")
ax1.grid(True)

# Bland-Altman plot
ax2.scatter(
    merged_df3["Fluxes"], differences,
    color='skyblue',
    edgecolors='darkblue',
    alpha=0.7,  # Set transparency
    s=150,  # Set marker size
    label='Differences'
)
ax2.axhline(0, color='red', linestyle='--', label='Zero Difference')
ax2.set_xlabel('Mean of Measurements')
ax2.set_ylabel('Difference (Flux_slit - Fluxes)')
ax2.set_title('Bland-Altman Plot')
ax2.set_ylim(-1e-12, 1e-12)
ax2.legend()
ax2.grid(True)

# Histogram
ax3.hist(differences, color='skyblue', bins=20, alpha=0.7)
ax3.set_xlabel('Difference (Flux_slit - Fluxes)')
ax3.set_ylabel('Frequency')
ax3.set_title('Difference Distribution')
ax3.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Create a scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(
    merged_df3["Fluxes"]/1e-12, merged_df3["Flux_slit"]/1e-12,
    color='skyblue',
    edgecolors='darkblue',
    alpha=0.7,  # Set transparency
    s=350,  # Set marker size
    label='PNe',
)

# Add error bars to the scatter plot
plt.errorbar(
    merged_df3["Fluxes"]/1e-12, merged_df3["Flux_slit"]/1e-12,
    xerr= merged_df3["Err Fluxes"]/1e-12, yerr=merged_df3["Constant_Err"]/1e-12,
    fmt='none',  # No markers for error bars
    ecolor='gray',  # Error bar color
    capsize=5,  # Size of error bar caps
)


# Add the identity line (y = x)
plt.plot([-3, 3], [-3, 3], color='red', linestyle='--', label='Identity Line')

# Set labels and title
plt.xlabel(r'F(H$\alpha$ + [NII]), Spec (VLT)',  fontsize=18)
plt.ylabel(r'F(H$\alpha$ + [NII]), SPLUS',  fontsize=18)
#plt.xlim(0.55e-13, 14e-13)
#plt.ylim(0.55e-13, 14e-13)
plt.xlim(0, 1.4)
plt.ylim(0, 1.4)
#plt.title('Comparison of Two Measurements')
plt.tick_params(axis='x', labelsize=18) 
plt.tick_params(axis='y', labelsize=18)

# Add legend
plt.legend(fontsize="x-large")

# Show plot
plt.grid(True)
plt.savefig("../compare-splus-spec.pdf")
plt.show()


In [ ]:
# Create a scatter plot
plt.figure(figsize=(12, 3))
# Bland-Altman plot
plt.scatter(
    merged_df3["Fluxes"], differences,
    color='skyblue',
    edgecolors='darkblue',
    alpha=0.7,  # Set transparency
    s=150,  # Set marker size
    label='Differences'
)
plt.axhline(0, color='red', linestyle='--', label='Zero Difference')
plt.ylim(-1e-12, 1e-12)


# Set labels and title
plt.xlabel(r'F(H$\alpha$ + [NII]), Spec (VLT)',  fontsize=18)
plt.ylabel(r'$\Delta$F(H$\alpha$ + [NII])',  fontsize=18)
plt.xlim(0.55e-13, 14e-13)

#plt.title('Comparison of Two Measurements')
plt.tick_params(axis='x', labelsize=18) 
plt.tick_params(axis='y', labelsize=18)

# Add legend
plt.legend(fontsize="x-large")

# Show plot
plt.grid(True)
plt.savefig("../compare-splus-spec-diference.pdf")
plt.show()